In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn import preprocessing, metrics
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import gc
from tqdm import tqdm
from scipy.sparse import csr_matrix
from  datetime import datetime, timedelta

In [2]:
cd C:\Users\minih\python_prac1\m5\git-repository\kaggle-m5-accuracy-library\library

C:\Users\minih\python_prac1\m5\git-repository\kaggle-m5-accuracy-library\library


In [3]:
from patair import reduce_mem_usage

In [4]:
cd C:\Users\minih\python_prac1\library\m5

C:\Users\minih\python_prac1\library\m5


In [5]:
product = pd.read_pickle("product.pickle").pipe(reduce_mem_usage)

Mem. usage decreased to  0.64 Mb (0.0% reduction)


In [19]:
datas = pd.read_pickle("all_sales_train_validation_translated_contain_null.pickle")

In [8]:
datas.shape

(22589864, 32)

In [7]:
check_feature=[
               'prev_28', 'p_prev_1','rolling_mean_t7', 
               'rolling_mean_t14', 'price_change_t1','rolling_std_t7', 'rolling_std_t14',
               'rolling_price_std_t7', 'rolling_price_std_t30', 'price_change_t1', 'rolling_price_std_t7',
               ]

In [8]:
#dataの時系列特徴列(prev_〇など)をみてnanを含むデータ削除する
#len(np.unique(datas.dropna(subset=check_feature)["id"]))
datas.dropna(subset=check_feature, inplace=True)
#len(np.unique(datas.dropna(subset=check_feature)["id"]))

In [9]:
#WRMSSEの計算用(kernelより)

NUM_ITEMS = 30490
DAYS_PRED = 28

weight_mat = np.c_[np.ones([NUM_ITEMS,1]).astype(np.int8), # level 1
                   pd.get_dummies(product.state_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.store_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.cat_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.dept_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.state_id.astype(str) + product.cat_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.state_id.astype(str) + product.dept_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.store_id.astype(str) + product.cat_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.store_id.astype(str) + product.dept_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.item_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.state_id.astype(str) + product.item_id.astype(str),drop_first=False).astype('int8').values,
                   np.identity(NUM_ITEMS).astype(np.int8) #item :level 12
                   ].T

weight_mat_csr = csr_matrix(weight_mat)
del weight_mat; gc.collect()

def weight_calc(data,product):
    
    # calculate the denominator of RMSSE, and calculate the weight base on sales amount

    sales_train_val = pd.read_csv('sales_train_validation.csv')

    d_name = ['d_' + str(i+1) for i in range(1913)]

    sales_train_val = weight_mat_csr * sales_train_val[d_name].values

    # calculate the start position(first non-zero demand observed date) for each item / 商品の最初の売上日
    # 1-1914のdayの数列のうち, 売上が存在しない日を一旦0にし、0を9999に置換。そのうえでminimum numberを計算
    df_tmp = ((sales_train_val>0) * np.tile(np.arange(1,1914),(weight_mat_csr.shape[0],1)))

    start_no = np.min(np.where(df_tmp==0,9999,df_tmp),axis=1)-1

    flag = np.dot(np.diag(1/(start_no+1)) , np.tile(np.arange(1,1914),(weight_mat_csr.shape[0],1)))<1

    sales_train_val = np.where(flag,np.nan,sales_train_val)

    # denominator of RMSSE / RMSSEの分母
    weight1 = np.nansum(np.diff(sales_train_val,axis=1)**2,axis=1)/(1913-start_no)

    # calculate the sales amount for each item/level
    df_tmp = data[(data['date'] > '2016-03-27') & (data['date'] <= '2016-04-24')]
    df_tmp['sell_num'] = df_tmp['sell_num'] * df_tmp['sell_price']
    df_tmp =df_tmp.groupby(['id'])['sell_num'].apply(np.sum)
    df_tmp = df_tmp[product.id].values
    
    weight2 = weight_mat_csr * df_tmp 

    weight2 = weight2/np.sum(weight2)

    del sales_train_val
    gc.collect()
    
    return weight1, weight2

#weight1, weight2 = weight_calc(datas,product)

def wrmsse(preds, data):
    
    # this function is calculate for last 28 days to consider the non-zero demand period
    
    # actual obserbed values / 正解ラベル
    y_true = data.get_label()
    
    y_true = y_true[-(NUM_ITEMS * DAYS_PRED):]
    preds = preds[-(NUM_ITEMS * DAYS_PRED):]
    # number of columns
    num_col = DAYS_PRED
    
    # reshape data to original array((NUM_ITEMS*num_col,1)->(NUM_ITEMS, num_col) ) / 推論の結果が 1 次元の配列になっているので直す
    print(preds)
    print(y_true)
    reshaped_preds = preds.reshape(num_col, NUM_ITEMS).T
    reshaped_true = np.array(y_true).reshape(num_col, NUM_ITEMS).T
    
          
    train = weight_mat_csr*np.c_[reshaped_preds, reshaped_true]
    print(train)
    
    score = np.sum(
                np.sqrt(
                    np.mean(
                        np.square(
                            train[:,:num_col] - train[:,num_col:])
                        ,axis=1) / weight1) * weight2)
    
    return 'wrmsse', score, False

def wrmsse_simple(preds, data):
    
    # actual obserbed values / 正解ラベル
    y_true = data.get_label()
    
    y_true = y_true[-(NUM_ITEMS * DAYS_PRED):]
    preds = preds[-(NUM_ITEMS * DAYS_PRED):]
    # number of columns
    num_col = DAYS_PRED
    
    # reshape data to original array((NUM_ITEMS*num_col,1)->(NUM_ITEMS, num_col) ) / 推論の結果が 1 次元の配列になっているので直す
    reshaped_preds = preds.reshape(num_col, NUM_ITEMS).T
    reshaped_true = y_true.reshape(num_col, NUM_ITEMS).T
          
    train = np.c_[reshaped_preds, reshaped_true]
   
    
    weight2_2 = weight2[:NUM_ITEMS]
    weight2_2 = weight2_2/np.sum(weight2_2)
    
    score = np.sum(
                np.sqrt(
                    np.mean(
                        np.square(
                            train[:,:num_col] - train[:,num_col:])
                        ,axis=1) /  weight1[:NUM_ITEMS])*weight2_2)
    
    return 'wrmsse', score, False

In [10]:
#weight_mat_csr
weights = pd.read_pickle("weight.pickle")
weight1 = weights[weights.index ==0]
weight2 = weights[weights.index ==1]

In [11]:
weight1=weight1.values
weight2=weight2.values

In [12]:
#x_train = datas[datas['date'] < '2016-04-18']
x_train = datas[datas['date'] < '2016-03-28']
y_train = x_train['sell_num']
#x_val = datas[datas['date'] >= '2016-04-18']
x_val = datas[(datas['date'] >= '2016-03-28')& (datas['date'] <= '2016-04-24')]
y_val = x_val['sell_num']
test = datas[datas['date'] > '2016-04-24']
del datas
gc.collect()

60

In [78]:
#test = datas[datas['date'] > '2016-04-24']
#del datas
#gc.collect()

2916

In [17]:
x_train.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sell_num', 'date', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'prev_7',
       'prev_14', 'prev_28', 'p_prev_1', 'rolling_mean_t7', 'rolling_mean_t30',
       'price_change_t1', 'rolling_std_t7', 'rolling_std_t30',
       'rolling_price_std_t7', 'rolling_price_std_t30', 'year', 'quarter',
       'month', 'week', 'day', 'dayofweek', 'is_year_end', 'is_year_start',
       'is_quarter_end', 'is_quarter_start', 'is_month_end', 'is_month_start',
       'is_weekend'],
      dtype='object')

In [13]:
features = [
   #訓練に用いる特徴量をここに記述
    "item_id",
    "dept_id",
    "cat_id",
    "store_id",
    "state_id",
    "event_name_1",
    #"event_type_1",
    "event_name_2",
    #"event_type_2",
    
    
    "snap_CA",
    "snap_TX",
    "snap_WI",
    
    "sell_price",
    # demand features.
    #'prev_1',
    #'prev_2',
    #'prev_7', 
    #'prev_14', 
    'prev_28',
    #'prev_182', 
    #'prev_365', 
    'p_prev_1', 
    'rolling_mean_t7',
    'rolling_mean_t14', 
    'rolling_mean_t30',
    'rolling_std_t7',
    'rolling_std_t14',
    'rolling_std_t30',
    #'rolling_mean_t60', 
    #'rolling_mean_t90',
    #'rolling_mean_t180',  
    # price features
    "price_change_t1",
    #"price_change_t365",
    #'rolling_price_max_t365', 
    #'price_change_t365',
    "rolling_price_std_t7",
    "rolling_price_std_t30",
    # time features.
    "year",
    "month",
    "week",
    "day",
    "dayofweek",
    #"is_year_end",
    #"is_year_start",
    #"is_quarter_end",
    #"is_quarter_start",
    #"is_month_end",
    #"is_month_start",
    #"is_weekend",
]

train_set = lgb.Dataset(x_train[features], y_train)
val_set = lgb.Dataset(x_val[features], y_val)

del x_train, y_train

In [14]:
val_set = lgb.Dataset(x_val[features], y_val)

In [15]:
results = pd.concat([pd.read_csv("result_lgb_CA.csv"),pd.read_csv("result_lgb_TX.csv"), pd.read_csv("result_lgb_WI.csv")])

In [16]:
results.drop("Unnamed: 0", inplace=True, axis=1)

In [18]:
results

,valid,predict
0,0.624452,0.703712
1,0.574923,0.613373
2,0.631345,0.619889
3,0.623523,0.603734
4,0.612145,0.787367
...,...,...
256111,0.095233,0.075415
256112,0.080732,0.076255
256113,0.092760,0.079591
256114,0.101607,0.083334


In [20]:
results["id"] = np.array(datas[datas["date"] >= "2016-4-25"]["id"])

In [21]:
del datas

In [22]:
results.to_csv("val_set_3.csv")

In [23]:
val_pred = results["valid"]
y_pred = results["predict"]
del results
gc.collect()

5606

In [24]:
a,b,c = wrmsse(np.array(val_pred), val_set)

[0.6244522  0.57492343 0.63134478 ... 0.09276033 0.1016068  0.09991039]
21405592    2.0
21436082    1.0
21466572    1.0
21497062    0.0
21527552    4.0
           ... 
22134302    0.0
22164792    0.0
22195282    0.0
22225772    0.0
22256262    0.0
Name: sell_num, Length: 853720, dtype: float16
[[5.65605042e+04 4.69861704e+04 3.36739699e+04 ... 4.35780000e+04
  4.66290000e+04 2.34330000e+04]
 [1.72510672e+04 2.85489592e+04 1.67719248e+04 ... 8.64400000e+03
  2.99740000e+04 1.33600000e+04]
 [1.86932324e+04 1.03923610e+04 3.95154036e+03 ... 1.26430000e+04
  1.21610000e+04 7.17600000e+03]
 ...
 [2.94277741e+00 3.64369358e+00 1.00308114e+00 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.98962047e+00 2.84166800e+00 1.09991359e+00 ... 2.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.85190655e+00 2.71451261e+00 1.00565088e+00 ... 5.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [25]:
b

0.7416319367098853

In [27]:
cd C:\Users\minih\python_prac1\m5\git-repository\kaggle-m5-accuracy-library\library

C:\Users\minih\python_prac1\m5\git-repository\kaggle-m5-accuracy-library\library


In [28]:
import metari

In [29]:
cd C:/Users/minih/python_prac1/library/m5/

C:\Users\minih\python_prac1\library\m5


In [30]:
df = pd.read_csv("sales_train_validation.csv")

In [31]:
calendar = pd.read_csv("calendar.csv")

In [32]:
prices = pd.read_csv("sell_prices.csv")

In [33]:
calendar[calendar["date"] == '2016-04-24']

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1912,2016-04-24,11613,Sunday,2,4,2016,d_1913,NaN,NaN,NaN,NaN,0,0,0


In [34]:
x_val["predict"] = np.array(val_pred)

In [35]:
x_val = pd.merge(product, x_val)

In [36]:
val_pred = np.array(x_val["predict"])

In [37]:
metari.calculate_WRMSSE(df, calendar, prices, 1, 1913, val_pred)

精度計算の前の前準備を実行中...
重みとRMSSEを計算中...



精度を計算中...


0.5605715993161652

In [41]:
test

,id,item_id,dept_id,cat_id,store_id,state_id,sell_num,date,event_name_1,event_type_1,...,price_change_t1,rolling_price_std_t7,rolling_price_std_t30,year,quarter,month,week,day,dayofweek,is_weekend
22259312,FOODS_1_001_CA_1_validation,0,0,0,0,0,NaN,2016-04-25,30,4,...,0.0,0.0,0.0,2016,2,4,17,25,0,0
22289802,FOODS_1_001_CA_1_validation,0,0,0,0,0,NaN,2016-04-26,30,4,...,0.0,0.0,0.0,2016,2,4,17,26,1,0
22320292,FOODS_1_001_CA_1_validation,0,0,0,0,0,NaN,2016-04-27,30,4,...,0.0,0.0,0.0,2016,2,4,17,27,2,0
22350782,FOODS_1_001_CA_1_validation,0,0,0,0,0,NaN,2016-04-28,30,4,...,0.0,0.0,0.0,2016,2,4,17,28,3,0
22381272,FOODS_1_001_CA_1_validation,0,0,0,0,0,NaN,2016-04-29,30,4,...,0.0,0.0,0.0,2016,2,4,17,29,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22988022,HOUSEHOLD_2_516_WI_3_validation,3048,6,2,9,2,NaN,2016-05-18,30,4,...,0.0,0.0,0.0,2016,2,5,20,18,2,0
23018512,HOUSEHOLD_2_516_WI_3_validation,3048,6,2,9,2,NaN,2016-05-19,30,4,...,0.0,0.0,0.0,2016,2,5,20,19,3,0
23049002,HOUSEHOLD_2_516_WI_3_validation,3048,6,2,9,2,NaN,2016-05-20,30,4,...,0.0,0.0,0.0,2016,2,5,20,20,4,0
23079492,HOUSEHOLD_2_516_WI_3_validation,3048,6,2,9,2,NaN,2016-05-21,30,4,...,0.0,0.0,0.0,2016,2,5,20,21,5,1


In [38]:
test["demand"] = np.array(y_pred)

In [39]:
test = pd.merge(product, test)

In [40]:
#提出用ファイルを読み込む
submission = pd.read_csv('sample_submission.csv')

In [41]:
#予測値を提出用ファイルに整形
predictions = test[['id', 'date', 'demand']]
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
evaluation = submission[submission['id'].isin(evaluation_rows)]

validation = submission[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])

In [97]:
final

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.599988,0.566160,0.686156,0.724494,0.839029,1.055731,0.814936,0.561176,0.614975,...,0.756827,0.840336,0.883184,0.646210,0.670912,0.682768,0.683275,0.771304,1.018370,0.991815
1,HOBBIES_1_002_CA_1_validation,0.251620,0.257284,0.257273,0.262017,0.326800,0.328736,0.410184,0.248644,0.294832,...,0.150187,0.174975,0.174359,0.115784,0.125014,0.177641,0.169470,0.205830,0.241133,0.242121
2,HOBBIES_1_003_CA_1_validation,0.220502,0.226776,0.226298,0.226599,0.273359,0.430842,0.386895,0.205200,0.235333,...,0.457358,0.614201,0.570673,0.365453,0.398210,0.355622,0.350138,0.475941,0.648376,0.595230
3,HOBBIES_1_004_CA_1_validation,2.236501,2.027887,1.931575,1.811769,2.236813,2.758679,3.303856,1.827533,1.948725,...,1.933203,2.603442,3.081599,2.069945,1.676970,1.608637,1.645808,2.156299,3.042337,2.957042
4,HOBBIES_1_005_CA_1_validation,1.026181,0.900697,1.046969,1.209807,1.230964,1.578332,2.038927,1.288779,1.259283,...,1.140608,1.444745,1.259563,0.934117,0.835920,0.899590,0.947127,1.169067,1.634534,1.837689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60976,FOODS_3_824_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60977,FOODS_3_825_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60978,FOODS_3_826_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [42]:
final

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.774747,0.735897,0.741893,0.770124,0.872003,1.004631,0.855211,0.678461,0.663533,...,0.833668,0.971490,0.928088,0.794373,0.738449,0.726964,0.716651,0.847944,1.044648,0.986663
1,HOBBIES_1_002_CA_1_validation,0.195891,0.184168,0.177030,0.179274,0.205673,0.252359,0.351508,0.246242,0.249211,...,0.124894,0.149744,0.160958,0.121690,0.125144,0.153716,0.152953,0.162121,0.186406,0.178195
2,HOBBIES_1_003_CA_1_validation,0.339161,0.333655,0.329177,0.338937,0.450694,0.636249,0.579297,0.294805,0.281723,...,0.403324,0.593757,0.630237,0.472883,0.420705,0.409192,0.410642,0.481249,0.613575,0.583141
3,HOBBIES_1_004_CA_1_validation,1.864629,1.643261,1.549690,1.520389,1.738896,2.604853,3.042477,1.869765,1.648368,...,1.890358,3.381522,3.348042,1.850506,1.599853,1.606589,1.551809,1.835364,2.986240,3.141442
4,HOBBIES_1_005_CA_1_validation,1.084863,0.941743,1.065647,1.149296,1.247388,1.667291,1.830696,1.296529,1.261945,...,1.252152,1.630657,1.525395,1.081352,1.007445,0.982698,0.988166,1.127923,1.507513,1.677911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60976,FOODS_3_824_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60977,FOODS_3_825_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60978,FOODS_3_826_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [43]:
#finalが提出形式のdfなので、これをto_csvする
final.to_csv('submission_cv3.csv', index = False)

In [46]:
final2 = pd.read_csv('submission_cv1.csv')
final3 = pd.read_csv('submission_cv3.csv')

In [71]:
final4 =  (final + final2 + final3)

In [72]:
final4.drop("id", axis=1, inplace=True)

In [73]:
final4 = final4/3

In [75]:
final4["id"] = np.array(final["id"])

In [77]:
final4.to_csv("submission_cv4.csv")